In [1]:
import pandas as pd
from pandas import Series,DataFrame
%pylab inline
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


Populating the interactive namespace from numpy and matplotlib


In [5]:
filename = 'Best RFM First 6 months.csv'
#dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M')
df_retail = pd.read_csv(filename,parse_dates=['InvoiceDate'])

#### Removing rows with CustomerID blank

In [6]:
## Removing rows with CustomerID blank
print (df_retail.shape)

df_retail = df_retail.drop(df_retail[df_retail['CustomerID'].isnull()].index)

print (df_retail.shape)

(34600, 9)
(34600, 9)


#### Adding a column "Total  Price" of the transaction

In [7]:
## adding a column total price of the transaction
df_retail['Total Price'] = df_retail['UnitPrice']* df_retail['Quantity']

#### Aggregating the details at Invoice level

In [ ]:
## Invoice level details

def dow(dt):
    return dt.day_name()

df_invoice = df_retail.groupby(['InvoiceNo','CustomerID', 'Country','InvoiceDate'])[['Total Price', 'Quantity']].sum()
print (df_invoice.shape)
df_invoice = pd.DataFrame(df_invoice.to_records())
df_invoice['Invoice_Month'] = df_invoice['InvoiceDate'].dt.strftime("%B")
df_invoice['DayofWeek'] = df_invoice['InvoiceDate'].map(dow)
print (df_invoice.shape)
df_invoice.head(5)


## Exploratory Data Analysis

#### Number of Orders by Month

In [ ]:
## Orders by Month

df_monthly = df_invoice.groupby('Invoice_Month')[['InvoiceNo']].count()
df_monthly = pd.DataFrame(df_monthly.to_records())
month_order = ["January", "February", "March", "April", "May", "June", "July","August", "September", "October", "November", "December"]
df_monthly.set_index("Invoice_Month").loc[month_order]['InvoiceNo'].plot(kind = 'bar')
xlabel('Month Name')
ylabel('Number of Orders')
title('Distribution of Orders by Month')

#### Orders by Day of the Week

In [ ]:
## Orders by Day of the Week

df_dow = df_invoice.groupby('DayofWeek')[['InvoiceNo']].count()
df_dow = pd.DataFrame(df_dow.to_records())
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df_dow.set_index("DayofWeek").loc[day_order]['InvoiceNo'].plot(kind = 'bar')
xlabel('Day of the Week')
ylabel('Number of Orders')
title('Distribution of Orders by Day of the Week')

#### Orders by Country

In [ ]:
## Orders by Country

df_country = df_invoice.groupby('Country')[['InvoiceNo']].count()
df_country = pd.DataFrame(df_country.to_records())
df_country.set_index("Country")['InvoiceNo'].plot(kind = 'bar')
xlabel('Country')
ylabel('Number of Orders')
title('Distribution of Orders by Country')

#### Aggregating Data at Customer Level for every month

In [ ]:
## Getting data at a customer level
## customer month firstorder of month last order of month total sales in month

## Sales and Quantity by Month
df_customer_sales = df_invoice.groupby(['CustomerID','Invoice_Month'])[['Total Price','Quantity']].sum()
df_customer_sales = pd.DataFrame(df_customer_sales.to_records())
#print (df_customer_sales.head(5))
#print (df_customer_sales.shape)

## Orders by Month
df_customer_orders = df_invoice.groupby(['CustomerID','Invoice_Month'])[['InvoiceNo']].count()
df_customer_orders = pd.DataFrame(df_customer_orders.to_records())
#print (df_customer_orders.head(5))
#print (df_customer_orders.shape)

## First order in a Month
df_customer_monthlyfirst = df_invoice.groupby(['CustomerID','Invoice_Month'])[['InvoiceDate']].min()
df_customer_monthlyfirst = pd.DataFrame(df_customer_monthlyfirst.to_records())
#print (df_customer_monthlyfirst.head(5))
#print (df_customer_monthlyfirst.shape)

## Last order in a Month
df_customer_monthlylast = df_invoice.groupby(['CustomerID','Invoice_Month'])[['InvoiceDate']].max()
df_customer_monthlylast = pd.DataFrame(df_customer_monthlylast.to_records())
#print (df_customer_monthlylast.head(5))
#print (df_customer_monthlylast.shape)

df_customer = df_customer_sales.merge(df_customer_monthlyfirst, on = ['CustomerID','Invoice_Month'],suffixes = ['_left','_right'])
#print (df_customer.head(5))
df_customer = df_customer.merge(df_customer_monthlylast, on = ['CustomerID','Invoice_Month'],suffixes = ['_firstorder','_lastorder'])
#print (df_customer.head(5))
df_customer = df_customer.merge(df_customer_orders, on = ['CustomerID','Invoice_Month'],suffixes = ['_firstorder','_lastorder'])
#print (df_customer.head(5))




df_customer.rename(columns = {'InvoiceDate_firstorder':'FirstOrder','InvoiceDate_lastorder':'LastOrder','InvoiceNo':'Ordercount'}, inplace = True) 
print (df_customer.head(5))
print (df_customer.shape)

#### Getting the RFM details of the customer

In [ ]:
from datetime import datetime

def recency(dt):
    dys = (datetime.strptime('2011-12-01', "%Y-%m-%d") - dt).days
    return dys


## RFM for 12 months - December 2010 - November 2011

## monetory value
df_customer_m = df_customer.groupby(['CustomerID'])[['Total Price']].sum()
df_customer_m = pd.DataFrame(df_customer_m.to_records())
df_customer_m.rename(columns = {'Total Price':'Monetory'}, inplace = True) 
#print (df_customer_m.head(5))


## frequency
df_customer_f = df_customer.groupby(['CustomerID'])[['Ordercount']].sum()
df_customer_f = pd.DataFrame(df_customer_f.to_records())
df_customer_f.rename(columns = {'Ordercount':'Frequency'}, inplace = True) 
#print (df_customer_f.head(5))

## recency

df_customer_r = df_customer.groupby(['CustomerID'])[['LastOrder']].max()
df_customer_r = pd.DataFrame(df_customer_r.to_records())
df_customer_r['Recency'] = df_customer_r['LastOrder'].map(recency)
df_customer_r = df_customer_r.drop(columns=['LastOrder'])
#print (df_customer_r.head(5))

## getting RFM together

df_customer_rfm = df_customer_m.merge(df_customer_f,on = ['CustomerID'])
df_customer_rfm = df_customer_rfm.merge(df_customer_r,on = ['CustomerID'])
print (df_customer_rfm.head(5))

#### Getting all important customer attributes together

In [ ]:
def getdays(s):
    return (s.days+1)

## Getting Aggregated values at customer level

## Total Sales, Orders, Items for the year
df_customer_ov = df_customer.groupby(['CustomerID'])[['Total Price','Quantity','Ordercount']].sum()
df_customer_ov = pd.DataFrame(df_customer_ov.to_records())
#print (df_customer_ov.head(5))

## First Order of Customer
df_customer_first = df_customer.groupby(['CustomerID'])[['FirstOrder']].min()
df_customer_first = pd.DataFrame(df_customer_first.to_records())
#print (df_customer_first.head(5))

## Last Order of Customer
df_customer_last = df_customer.groupby(['CustomerID'])[['LastOrder']].max()
df_customer_last = pd.DataFrame(df_customer_last.to_records())
#print (df_customer_last.head(5))

## Number of active months
df_customer_months = df_customer.groupby(['CustomerID'])[['Invoice_Month']].count()
df_customer_months = pd.DataFrame(df_customer_months.to_records())
df_customer_months.rename(columns = {'Invoice_Month':'ActiveMonths'}, inplace = True) 
#print (df_customer_months.head(5))


## Days between first and last order
df_customer_daysbetween = df_customer_first.merge(df_customer_last, on = 'CustomerID')
df_customer_daysbetween['ActiveDays'] = (df_customer_daysbetween['LastOrder'] - df_customer_daysbetween['FirstOrder']).map(getdays)
#print (df_customer_daysbetween.head(5))


In [ ]:
## Getting all important customer attributes together

df_customer_att = df_customer_ov.merge(df_customer_months, on = 'CustomerID')
df_customer_att = df_customer_att.merge(df_customer_daysbetween, on ='CustomerID')
df_customer_att = df_customer_att.drop(columns=['LastOrder','FirstOrder'])
df_customer_att = df_customer_att.merge(df_customer_r,on = 'CustomerID')
df_customer_att.rename(columns = {'Quantity':'Total Items','Ordercount': 'Total Orders','Total Price':'Total Sales'}, inplace = True)
df_customer_att['Average Basket'] = round(df_customer_att['Total Sales']/ df_customer_att['Total Orders'])
df_customer_att.head(5)

### Cleaning the Data

#### Removing customers with negative sales

In [ ]:
## Removing customers with negative sales

print (df_customer_att.shape)
print (df_customer_rfm.shape)

df_customer_att = df_customer_att[df_customer_att['Total Sales'] > 0]
df_customer_rfm = df_customer_rfm[df_customer_rfm['Monetory'] > 0 ]

print (df_customer_att.shape)
print (df_customer_rfm.shape)

In [ ]:
for col in ['Total Sales','Total Items','Total Orders','Average Basket']:
    print (col)

#### Removing the outliers

In [ ]:
## Removing the outliers



print (df_customer_att.describe())
print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
print (df_customer_rfm.describe())
print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')


for col in ['Total Sales','Total Items','Total Orders','Average Basket']:

    mn = df_customer_att[col].mean()
    sd = df_customer_att[col].std()
    ## Checking if the that column has outliers
    def isnotoutlier(n):
        if n > (mn+ (3* sd)):
            return False
        elif n < (mn - (3* sd)):
            return False
        else:
            return True

    is_notoutlier = df_customer_att[col].map(isnotoutlier)

    df_customer_att = df_customer_att[is_notoutlier]
    df_customer_rfm = df_customer_rfm[is_notoutlier]


print (df_customer_att.describe())
print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
print (df_customer_rfm.describe())
print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

In [ ]:
print (df_customer_att.shape,df_customer_rfm.shape)
print(df_customer_att.head(5))

## CLUSTERING

#### Over all customer attributes

In [ ]:
## Clustering Based on Customer attributes

from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

# creating dataframe without customerid for clustering
df_temp = df_customer_att.drop(columns=['CustomerID', 'ActiveMonths'])

# getting the z-values and storing the mean and standard deviations to unscale them whenever needed

means = []
stds = []
i = 0
df_scaled = pd.DataFrame()
for var in df_temp.columns:
    means.append(df_temp[var].mean())
    stds.append(df_temp[var].std())
    df_scaled[var] = (df_temp[var] - means[i])/stds[i]
    i = i+ 1

print (df_scaled.head(5))

# Clustering
data = df_scaled.values[:,:]

sum_of_distances = {}
for num_clusters in range(1,10):
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters, random_state=1,n_init = 50)
    kmeans.fit(data)
    sum_of_distances[num_clusters] = kmeans.inertia_
Series(sum_of_distances).sort_index().plot()
xlabel('Number of clusters')
ylabel('Sum of distances of points to closest cluster centers')

In [ ]:
## clustering

kmeans = KMeans(init='k-means++', n_clusters=4, random_state=1)
kmeans.fit(data)
cluster_centers = kmeans.cluster_centers_
print (cluster_centers)

df_temp['OverallCluster'] = kmeans.predict(df_scaled)
df_customer_att['OverallCluster'] = kmeans.predict(df_scaled)
print (df_temp['OverallCluster'].value_counts())


In [ ]:
## Unscaling the cluster coefficients
import copy
m = 0
n = 0
cluster_centers_org = copy.deepcopy(cluster_centers)


for m in range(len(cluster_centers)):
    for n in range(len(cluster_centers[m])):
        cluster_centers_org[m][n] = (cluster_centers[m][n]*stds[n])+means[n]

# for x in range(len(cluster_centers_org)):
#     print (cluster_centers_org[x])

In [ ]:
df_scaled.head(5)

In [ ]:
scatter(df_customer_att['Total Sales'],df_customer_att['Recency'], c=df_customer_att['OverallCluster'], cmap='Set3')
plt.xlabel('Total Sales')
plt.ylabel('Recency')

#### Clustering based on RFM Segments

In [ ]:
## Clusters Based on RFM segments

df_rfm_temp = df_customer_rfm.drop(columns=['CustomerID'])

# #Scaling the columns

# means = []
# stds = []
# i = 0
# df_scaled = pd.DataFrame()
# for var in df_temp.columns:
#     means.append(df_temp[var].mean())
#     stds.append(df_temp[var].std())
#     df_scaled[var] = (df_temp[var] - means[i])/stds[i]
#     i = i+ 1


df_rfm_scaled = df_rfm_temp.apply(lambda x: (x - np.mean(x)) / np.std(x), axis=0)


rfm_data = df_rfm_scaled.values[:,:]

sum_of_distances = {}
for num_clusters in range(1,10):
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters, random_state=1)
    kmeans.fit(rfm_data)
    sum_of_distances[num_clusters] = kmeans.inertia_
Series(sum_of_distances).sort_index().plot()
xlabel('Number of clusters')
ylabel('Sum of distances of points to closest cluster centers')

# Clustering
kmeans = KMeans(init='k-means++', n_clusters=4, random_state=1)
kmeans.fit(rfm_data)
rfm_cluster_centers = kmeans.cluster_centers_
print (rfm_cluster_centers)
df_customer_rfm['RFMCluster'] = kmeans.predict(rfm_data)

print (df_customer_rfm['RFMCluster'].value_counts())



In [ ]:
df_rfm_temp.head(5)

In [ ]:
scatter(df_customer_rfm['Monetory'], df_customer_rfm["Recency"], c=df_customer_rfm['RFMCluster'], cmap='Set3')
plt.xlabel('Total Sales')
plt.ylabel('Recency')

#### Based on Monetory Value

In [ ]:

#Clusters on Monetory value

monetory_data = df_customer_rfm[['Monetory']]

sum_of_distances = {}
for num_clusters in range(1,10):
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters, random_state=1)
    kmeans.fit(monetory_data)
    sum_of_distances[num_clusters] = kmeans.inertia_
Series(sum_of_distances).sort_index().plot()
xlabel('Number of clusters')
ylabel('Sum of distances of points to closest cluster centers')

# Clustering
kmeans = KMeans(init='k-means++', n_clusters=4, random_state=1)
kmeans.fit(monetory_data)
monetory_cluster_centers = kmeans.cluster_centers_
print (monetory_cluster_centers)
df_customer_rfm['MonetoryCluster'] = kmeans.predict(monetory_data)

print (df_customer_rfm['MonetoryCluster'].value_counts())

## Sorting the cluster labels
idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(4)


df_customer_rfm['MonetoryCluster'] = lut[kmeans.labels_]
df_customer_rfm['MonetoryCluster'] = df_customer_rfm['MonetoryCluster'].apply(lambda x:x+1)
print (df_customer_rfm['MonetoryCluster'].value_counts())


In [ ]:
plt.scatter(df_customer_rfm['MonetoryCluster'],df_customer_rfm['Monetory'])

#### Based on Frequency

In [ ]:
#Clusters on Frequency value

frequency_data = df_customer_rfm[['Frequency']]

sum_of_distances = {}
for num_clusters in range(1,10):
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters, random_state=1)
    kmeans.fit(frequency_data)
    sum_of_distances[num_clusters] = kmeans.inertia_
Series(sum_of_distances).sort_index().plot()
xlabel('Number of clusters')
ylabel('Sum of distances of points to closest cluster centers')

# Clustering
kmeans = KMeans(init='k-means++', n_clusters=4, random_state=1)
kmeans.fit(frequency_data)
frequency_cluster_centers = kmeans.cluster_centers_
print (frequency_cluster_centers)
df_customer_rfm['FrequencyCluster'] = kmeans.predict(frequency_data)
print (df_customer_rfm['FrequencyCluster'].value_counts())

## Sorting the cluster labels
idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(4)

df_customer_rfm['FrequencyCluster'] = lut[kmeans.labels_]
df_customer_rfm['FrequencyCluster'] = df_customer_rfm['FrequencyCluster'].apply(lambda x:x+1)
print (df_customer_rfm['FrequencyCluster'].value_counts())



In [ ]:
plt.scatter(df_customer_rfm['FrequencyCluster'],df_customer_rfm['Frequency'])

#### Based on Recency

In [ ]:
#Clusters on Recency value

recency_data = df_customer_rfm[['Recency']]

sum_of_distances = {}
for num_clusters in range(1,10):
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters, random_state=1)
    kmeans.fit(recency_data)
    sum_of_distances[num_clusters] = kmeans.inertia_
Series(sum_of_distances).sort_index().plot()
xlabel('Number of clusters')
ylabel('Sum of distances of points to closest cluster centers')

# Clustering
kmeans = KMeans(init='k-means++', n_clusters=4, random_state=1)
kmeans.fit(recency_data)
recency_cluster_centers = kmeans.cluster_centers_
print (recency_cluster_centers)
df_customer_rfm['RecencyCluster'] = kmeans.predict(recency_data)

print(df_customer_rfm['RecencyCluster'].value_counts())

## Sorting the cluster labels
idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(3,-1,-1)


df_customer_rfm['RecencyCluster'] = lut[kmeans.labels_]
df_customer_rfm['RecencyCluster'] = df_customer_rfm['RecencyCluster'].apply(lambda x:x+1)
print(df_customer_rfm['RecencyCluster'].value_counts())

In [ ]:
plt.scatter(df_customer_rfm['RecencyCluster'],df_customer_rfm['Recency'])

### Getting the RFM segments together

In [ ]:
df_customer_rfm['RFMCombined'] = (df_customer_rfm['RecencyCluster']*100)+((df_customer_rfm['FrequencyCluster']*10))+df_customer_rfm['MonetoryCluster']
print (df_customer_rfm.head(5))
df_customer_rfm = df_customer_rfm.drop(columns=['RFMCluster','RecencyCluster','FrequencyCluster','MonetoryCluster'])
print(df_customer_rfm.head(5))

In [ ]:
scatter(df_customer_rfm['Monetory'], df_customer_rfm["Recency"], c=df_customer_rfm['RFMCombined'], cmap='Set3')
plt.xlabel('Total Sales')
plt.ylabel('Recency')

## Market Basket

In [9]:
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [10]:
filename = 'Best RFM First 6 months.csv'
#dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M')
df_apriori = pd.read_csv(filename)

In [11]:
print(df_apriori.shape)
print(df_apriori.head(5))

(34600, 9)
  InvoiceNo StockCode                        Description  Quantity  \
0    536367     84879      ASSORTED COLOUR BIRD ORNAMENT        32   
1    536367     22745         POPPY'S PLAYHOUSE BEDROOM          6   
2    536367     22748          POPPY'S PLAYHOUSE KITCHEN         6   
3    536367     22749  FELTCRAFT PRINCESS CHARLOTTE DOLL         8   
4    536367     22310            IVORY KNITTED MUG COSY          6   

   InvoiceDate  UnitPrice  CustomerID         Country  RFM  
0  40513.35694       1.69       13047  United Kingdom   11  
1  40513.35694       2.10       13047  United Kingdom   11  
2  40513.35694       2.10       13047  United Kingdom   11  
3  40513.35694       3.75       13047  United Kingdom   11  
4  40513.35694       1.65       13047  United Kingdom   11  


In [12]:
## Cleaning the data - Removing nulls and returns 

df_apriori.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df_apriori['InvoiceNo'] = df_apriori['InvoiceNo'].astype('str')
df_apriori = df_apriori[~df_apriori['InvoiceNo'].str.contains('C')]

In [13]:
## Cleaning the description - removing spaces, nulls, bad entries

print('Initial shape:', df_apriori.shape)
df_apriori.dropna(axis = 0,subset = ['Description'],inplace = True)
print('Removed Nulls:', df_apriori.shape)

df_apriori = df_apriori[df_apriori['Description'].str.len() > 10]
print('Removed bad entries:', df_apriori.shape)

df_apriori['Description'] = df_apriori['Description'].str.strip()
print('Stripped the spaces:', df_apriori.shape)


def check(s):
    cnt = 0
    for word in ['?','damage','Damage','wrong','Wrong','thrown','Thrown','postage','Postage']:
        if  word in s:
            cnt = 1
    if cnt == 1:
        return False
    else:
        return True

df_apriori = df_apriori[df_apriori['Description'].map(check)]
print('Furthur Removed bad entries:', df_apriori.shape)

Initial shape: (33805, 9)
Removed Nulls: (33805, 9)
Removed bad entries: (33596, 9)
Stripped the spaces: (33596, 9)
Furthur Removed bad entries: (33596, 9)


In [14]:
basket = (df_apriori
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket.head(1)


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)


In [33]:
## Getting itemsets with support > 5%

frequent_itemsets = apriori(basket_sets, min_support=0.009, use_colnames=True)


In [34]:
frequent_itemsets.shape

(2223, 2)

In [46]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
#rules.sort_values(by=['lift'],ascending = False).head(30)

rules[ (rules['lift'] >= 40) & (rules['confidence'] >= 0.7) ].sort_values(by = ['lift'],ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1881,(SET/6 COLLAGE PAPER PLATES),(SET/6 COLLAGE PAPER CUPS),0.009340,0.010674,0.009340,1.000000,93.687500,0.009240,inf
1880,(SET/6 COLLAGE PAPER CUPS),(SET/6 COLLAGE PAPER PLATES),0.010674,0.009340,0.009340,0.875000,93.687500,0.009240,7.925284
1461,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.010674,0.010674,0.009340,0.875000,81.976563,0.009226,7.914610
1460,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.010674,0.010674,0.009340,0.875000,81.976563,0.009226,7.914610
4582,"(CHILDS GARDEN FORK PINK, CHILDS GARDEN TROWEL...","(CHILDS GARDEN TROWEL PINK, CHILDS GARDEN FORK...",0.012675,0.011341,0.010674,0.842105,74.253870,0.010530,6.261508
4583,"(CHILDS GARDEN TROWEL BLUE, CHILDS GARDEN FORK...","(CHILDS GARDEN TROWEL PINK, CHILDS GARDEN FORK...",0.012675,0.011341,0.010674,0.842105,74.253870,0.010530,6.261508
4579,"(CHILDS GARDEN TROWEL PINK, CHILDS GARDEN FORK...","(CHILDS GARDEN FORK PINK, CHILDS GARDEN TROWEL...",0.011341,0.012675,0.010674,0.941176,74.253870,0.010530,16.784523
4578,"(CHILDS GARDEN TROWEL PINK, CHILDS GARDEN FORK...","(CHILDS GARDEN TROWEL BLUE, CHILDS GARDEN FORK...",0.011341,0.012675,0.010674,0.941176,74.253870,0.010530,16.784523
533,(GARDENERS KNEELING PAD KEEP CALM),(GARDENERS KNEELING PAD CUP OF TEA),0.011341,0.012008,0.010007,0.882353,73.480392,0.009870,8.397932
532,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.012008,0.011341,0.010007,0.833333,73.480392,0.009870,5.931955


In [47]:
rules[ (rules['lift'] >= 40) & (rules['confidence'] >= 0.7) ].to_csv('MarketBasketGood.csv')

In [22]:
## Blue green ivory pink black 
## for each two products, check if all words except two are similar, then remove the unsimilar word and attach the remaining together

In [23]:
df_products = pd.DataFrame(df_apriori['Description'].value_counts()).reset_index()
print(df_products.head(5))
print(df_products.shape)

                                index  Description
0  WHITE HANGING HEART T-LIGHT HOLDER          230
1            REGENCY CAKESTAND 3 TIER          200
2             LUNCH BAG RED RETROSPOT          166
3                       PARTY BUNTING          151
4       ASSORTED COLOUR BIRD ORNAMENT          134
(2552, 2)


In [24]:
## Jugaaaad

def jugaad(s):
    s = s.strip()
    new_word = ''
    words_s = s.split(' ')
    for product in df_products['index'].value_counts().index.to_list():
        product = product.strip()
        words_p = product.split(' ')
        similar_words = [i for i, j in zip(words_s, words_p) if i == j]
        if len(similar_words) == (len(words_s)-1):
            new_word = ' '.join(similar_words)
            return product
    return s
#     if len(new_word) > 2:
#         return new_word
#     else:
#         return s
            

def prolevel(s):
    s = s.strip()
    new_word = ''
    words_s = s.split(' ')
    for product in df_products['index'].value_counts().index.to_list():
        product = product.strip()
        words_p = product.split(' ')
        similar_words = [i for i, j in zip(words_s, words_p) if i == j]
        if len(similar_words) == (len(words_s)-1):
            new_word = ' '.join(similar_words)
            return new_word
    return s
#     if len(new_word) > 2:
#         return new_word
#     else:
#         return s

    

df_products['Similar Products'] = df_products['index'].map(jugaad)
print(df_products.shape)
df_products['New Description'] = df_products['index'].map(prolevel)

(2552, 3)


In [25]:
df_products.head(5)
df_products = df_products.drop(columns=['Description'])
df_products.rename(columns = {'index':'Description'}, inplace = True) 
df_products.head(5)


,Description,Similar Products,New Description
0,WHITE HANGING HEART T-LIGHT HOLDER,RED HANGING HEART T-LIGHT HOLDER,HANGING HEART T-LIGHT HOLDER
1,REGENCY CAKESTAND 3 TIER,SWEETHEART CAKESTAND 3 TIER,CAKESTAND 3 TIER
2,LUNCH BAG RED RETROSPOT,JUMBO BAG RED RETROSPOT,BAG RED RETROSPOT
3,PARTY BUNTING,PARTY PIZZA DISH BLUE POLKADOT,PARTY
4,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR BIRD ORNAMENT


In [26]:
df_apriori.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,RFM
0,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,40513.35694,1.69,13047,United Kingdom,11
1,536367,22745,POPPY'S PLAYHOUSE BEDROOM,6,40513.35694,2.10,13047,United Kingdom,11
2,536367,22748,POPPY'S PLAYHOUSE KITCHEN,6,40513.35694,2.10,13047,United Kingdom,11
3,536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,40513.35694,3.75,13047,United Kingdom,11
4,536367,22310,IVORY KNITTED MUG COSY,6,40513.35694,1.65,13047,United Kingdom,11


In [27]:
df_apriori_new = df_apriori.merge(df_products, on = ['Description'] )
print(df_apriori_new.head(5))
df_apriori_new = df_apriori_new.drop(columns=['Description','Similar Products'])
print(df_apriori_new.head(5))

  InvoiceNo StockCode                    Description  Quantity  InvoiceDate  \
0    536367     84879  ASSORTED COLOUR BIRD ORNAMENT        32  40513.35694   
1    536536     84879  ASSORTED COLOUR BIRD ORNAMENT        80  40513.57292   
2    536946     84879  ASSORTED COLOUR BIRD ORNAMENT        16  40515.51944   
3    537137     84879  ASSORTED COLOUR BIRD ORNAMENT        13  40517.52986   
4    537204     84879  ASSORTED COLOUR BIRD ORNAMENT         8  40517.61458   

   UnitPrice  CustomerID         Country  RFM               Similar Products  \
0       1.69       13047  United Kingdom   11  ASSORTED COLOUR BIRD ORNAMENT   
1       1.69       18144  United Kingdom   11  ASSORTED COLOUR BIRD ORNAMENT   
2       1.69       13013  United Kingdom   10  ASSORTED COLOUR BIRD ORNAMENT   
3       1.69       16327  United Kingdom   11  ASSORTED COLOUR BIRD ORNAMENT   
4       1.69       15555  United Kingdom   11  ASSORTED COLOUR BIRD ORNAMENT   

                 New Description  
0  ASSORT

In [28]:
basket_new = (df_apriori_new
          .groupby(['InvoiceNo', 'New Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_new.head(1)

New Description,DESIGN COTTON TOTE BAG,STICKERS,WRITING SET,& WHITE BREAKFAST TRAY,1 WICK MORRIS BOXED CANDLE,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,ZINC HEART T-LIGHT HOLDER,ZINC HEART WALL ORGANISER,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC T-LIGHT HOLDER LARGE,ZINC T-LIGHT HOLDER STARS,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets_new = basket_new.applymap(encode_units)


In [30]:
## Getting itemsets with support > 5%

frequent_itemsets = apriori(basket_sets_new, min_support=0.02, use_colnames=True)
print(frequent_itemsets.shape)

(480, 2)


In [31]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [32]:
rules.sort_values(by=['lift'],ascending = False).head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
316,(WHITE SPOT CERAMIC DRAWER KNOB),(STRIPE CERAMIC DRAWER KNOB),0.030687,0.031354,0.020680,0.673913,21.493525,0.019718,2.970514
317,(STRIPE CERAMIC DRAWER KNOB),(WHITE SPOT CERAMIC DRAWER KNOB),0.031354,0.030687,0.020680,0.659574,21.493525,0.019718,2.847357
321,(WATERING CAN PINK BUNNY),(WATERING CAN BLUE ELEPHANT),0.032021,0.036024,0.022682,0.708333,19.662809,0.021528,3.305061
320,(WATERING CAN BLUE ELEPHANT),(WATERING CAN PINK BUNNY),0.036024,0.032021,0.022682,0.629630,19.662809,0.021528,2.613542
323,(WOOD S/3 CABINET ANT WHITE FINISH),(WOOD 2 DRAWER CABINET WHITE FINISH),0.043362,0.031354,0.022682,0.523077,16.682815,0.021322,2.031031
322,(WOOD 2 DRAWER CABINET WHITE FINISH),(WOOD S/3 CABINET ANT WHITE FINISH),0.031354,0.043362,0.022682,0.723404,16.682815,0.021322,3.458613
319,(TRAVEL CARD WALLET KEEP CALM),(TRAVEL CARD WALLET),0.033356,0.038692,0.021348,0.640000,16.540690,0.020057,2.670299
318,(TRAVEL CARD WALLET),(TRAVEL CARD WALLET KEEP CALM),0.038692,0.033356,0.021348,0.551724,16.540690,0.020057,2.156361
312,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4),0.044029,0.046698,0.028019,0.636364,13.627273,0.025963,2.621581
313,(ROUND SNACK BOXES SET OF 4),(ROUND SNACK BOXES SET OF4 WOODLAND),0.046698,0.044029,0.028019,0.600000,13.627273,0.025963,2.389927
